In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os

In [6]:
!ls data/fraud-detection

models	tmp  train.csv	train_sample.csv


In [3]:
dtypes = {
            'ip'            : 'uint32',
            'app'           : 'uint16',
            'device'        : 'uint16',
            'os'            : 'uint16',
            'channel'       : 'uint16',
            'is_attributed' : 'uint8',
            'click_id'      : 'uint32',
            }

In [4]:
train_df = pd.read_csv('data/fraud-detection/train_sample.csv', parse_dates=['click_time'], \
                       dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

In [5]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [6]:
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')

In [16]:
def do_count( df, group_cols, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return( df )

In [19]:
train_df = do_count( train_df, ['ip', 'day', 'hour'], 'ip_tcount', show_max=True ); gc.collect()

Aggregating by  ['ip', 'day', 'hour'] ...
ip_tcount max value =  28


0

In [20]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_tcount
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,7,1
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,4
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,7,1
3,94584,13,1,13,477,2017-11-07 04:58:08,0,4,7,1
4,68413,12,1,1,178,2017-11-09 09:00:09,0,9,9,1


In [29]:
group_cols = ['ip', 'app']
train_df[group_cols].groupby(group_cols).size().rename('ip_app_count').to_frame().reset_index()

,ip,app,ip_app_count
0,9,9,1
1,10,11,1
2,10,12,1
3,10,18,1
4,19,14,1
5,20,2,2
6,20,12,1
7,20,18,1
8,25,3,1
9,27,1,1


In [32]:
train_df.head(3)

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_tcount
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,7,1
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,4
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,7,1


In [85]:
train_df['ip_click_time'] = train_df[['ip', 'click_time']].groupby(['ip']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds

,click_time
0,61540.0
1,12747.0
2,57936.0
3,19577.0
4,51097.0
5,13239.0
6,21269.0
7,52342.0
8,1648.0
9,NaN


In [39]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_tcount,ip_click_time
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,7,1,61540.0
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,4,12747.0
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,7,1,57936.0
3,94584,13,1,13,477,2017-11-07 04:58:08,0,4,7,1,19577.0
4,68413,12,1,1,178,2017-11-09 09:00:09,0,9,9,1,51097.0


In [77]:
train_df['ip_click_time_past'] = train_df[['ip', 'click_time']].groupby(['ip']).click_time.transform(lambda x: x.diff().shift(+1)).dt.seconds

In [78]:
train_df.tail(5)

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_click_time,ip_click_time_past
99995,124883,11,1,19,122,2017-11-09 13:25:41,0,13,9,NaN,11023.0
99996,85150,9,1,13,244,2017-11-07 11:25:43,0,11,7,NaN,20439.0
99997,18839,3,1,13,19,2017-11-08 11:38:42,0,11,8,NaN,43445.0
99998,114276,15,1,12,245,2017-11-08 17:55:21,0,17,8,NaN,61896.0
99999,119349,14,1,15,401,2017-11-07 14:32:27,0,14,7,NaN,181.0


Let's replace NaN with the group median.
We could compare performance with using something like -999 (since trees split on values and this value will never be seen in data set, this is a safe way to create an unknown category to split on). 

In [49]:
from scipy.stats import gmean

In [83]:
train_df.groupby("ip")["ip_click_time"].transform(lambda x: x.fillna(x.median())).to_frame().tail()

,ip_click_time
99995,8861.5
99996,40960.0
99997,43445.0
99998,45093.0
99999,50377.0


In [67]:
train_df.tail(5)

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_tcount,ip_click_time,ip_click_time_past
99995,124883,11,1,19,122,2017-11-09 13:25:41,0,13,9,1,<bound method Series.median of 33228 4238....,11023.0
99996,85150,9,1,13,244,2017-11-07 11:25:43,0,11,7,1,<bound method Series.median of 597 67744....,20439.0
99997,18839,3,1,13,19,2017-11-08 11:38:42,0,11,8,1,<bound method Series.median of 7050 54090....,43445.0
99998,114276,15,1,12,245,2017-11-08 17:55:21,0,17,8,2,<bound method Series.median of 2510 56037....,61896.0
99999,119349,14,1,15,401,2017-11-07 14:32:27,0,14,7,4,<bound method Series.median of 1269 37799....,181.0


In [89]:
train_df.head(3)

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_click_time,ip_click_time_past
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,7,61540.0,NaN
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,12747.0,NaN
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,7,57936.0,NaN


In [7]:

def do_click_shift(df, group_cols, shift_name, agg_type='float32', shift_val=-1, show_max=False, \
                   show_agg=True):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    #gp = 
    df[shift_name] = df[group_cols + ['click_time']].groupby(group_cols).click_time.transform(lambda x: x.diff().shift(shift_val)).dt.seconds#.rename(shift_name).to_frame().reset_index()
    #add fillna code from above here
    #df = df.merge(gp, on=group_cols, how='left')
    #del gp
    if show_max:
        print( shift_name + " max value = ", df[shift_name].max() )
    df[shift_name] = df[shift_name].astype(agg_type)
    gc.collect()
    return( df )

will there still be NAs after filling with median? if so, could use -999 value

In [8]:
do_click_shift(train_df, ['ip'], 'ip_click_time2')

Aggregating by  ['ip'] ...


,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_click_time2
0,87540,12,1,13,497,2017-11-07 09:30:38,0,9,7,61540.0
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,12747.0
2,101424,12,1,19,212,2017-11-07 18:05:24,0,18,7,57936.0
3,94584,13,1,13,477,2017-11-07 04:58:08,0,4,7,19577.0
4,68413,12,1,1,178,2017-11-09 09:00:09,0,9,9,51097.0
5,93663,3,1,17,115,2017-11-09 01:22:13,0,1,9,13239.0
6,17059,1,1,17,135,2017-11-09 01:17:58,0,1,9,21269.0
7,121505,9,1,25,442,2017-11-07 10:01:53,0,10,7,52342.0
8,192967,2,2,22,364,2017-11-08 09:35:17,0,9,8,1648.0
9,143636,3,1,19,135,2017-11-08 12:35:26,0,12,8,NaN


In [9]:
train_df.loc[train_df['ip']==105560].head()

,ip,app,device,os,channel,click_time,is_attributed,hour,day,ip_click_time2
1,105560,25,1,17,259,2017-11-07 13:40:27,0,13,7,12747.0
1928,105560,15,1,19,245,2017-11-07 17:12:54,0,17,7,24613.0
2065,105560,26,1,19,121,2017-11-09 00:03:07,0,0,9,44767.0
2966,105560,20,1,22,478,2017-11-08 12:29:14,0,12,8,9104.0
2994,105560,3,1,13,466,2017-11-08 15:00:58,0,15,8,68187.0


### This is almost working but need this index column to be the group (here: ip) value:

In [103]:
train_df[['ip'] + ['click_time']].groupby(['ip']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds.reset_index().rename(columns={'click_time':'ip_click_time2'})

,index,ip_click_time2
0,0,61540.0
1,1,12747.0
2,2,57936.0
3,3,19577.0
4,4,51097.0
5,5,13239.0
6,6,21269.0
7,7,52342.0
8,8,1648.0
9,9,NaN


In [97]:
df2 = train_df.merge(gp, on=['ip'], how='left')
df2.head()

KeyError: 'ip'

In [99]:
gp.head(2)

,index,ip_click_time2
0,0,61540.0
1,1,12747.0
